In [13]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.model_selection import train_test_split
from os import path

In [19]:
dataset_path = path.abspath('../') + "/datasets"
data = pd.read_csv(dataset_path + '/spam.csv')

In [15]:
def preprocessing(text):
    # case folding
    text = text.lower()

    # replace numbers with token
    text = re.sub(r'\d+', ' <NUM> ', text)

    # remove punctuation (tapi jangan spasi)
    text = re.sub(r'[^\w\s]', '', text)

    # stopword removal (tapi jangan terlalu agresif)
    factory = StopWordRemoverFactory()
    stopwords = set(factory.get_stop_words())
    words = text.split()
    text = " ".join([word for word in words if word not in stopwords])

    return text

In [16]:
data['Pesan'] = data['Pesan'].astype(str).apply(preprocessing)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data['Pesan'], data['Kategori'], test_size=0.2, random_state=42, stratify=data['Kategori'])

train_df = pd.DataFrame({'Pesan': X_train, 'Kategori': y_train})
test_df = pd.DataFrame({'Pesan': X_test, 'Kategori': y_test})

train_df.to_csv(dataset_path + "/train.csv", index=False)
test_df.to_csv(dataset_path + "/test.csv", index=False)